<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [143]:
import requests
from bs4 import BeautifulSoup as bs
import re
from pprint import pprint
import pandas as pd

In [144]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
}

url = 'https://kazan.hh.ru/search/vacancy?text=data+scientist&area='

Находим число страниц в пагинации, то есть общее число страниц с результатами поиска

In [145]:
response = requests.get(url, headers=headers)
soup = bs(response.text, 'html.parser')
max_page = int(soup.find_all('a', attrs={'class': ['bloko-button']})[-2].text)

In [146]:
vacancies_list = []

Код парсера. Проходимся по всем страницам, считываем параметры вакансий и сохраняем все вакансии в виде списка словарей. Если зарплата представлена в виде 100 - 200 руб., то заполняется минимальная и максимальная зарплата. Если "от 100 руб", то заполняется минимальная зарплата. Если "до 100 руб", то заполняется максимальная зарплата.

In [147]:
for page_num in range(max_page):
    response = requests.get(f'https://kazan.hh.ru/search/vacancy?text=data+scientist&area=&page={page_num}', 
                            headers=headers)
    soup = bs(response.text, 'html.parser')
    
    vacancies = soup.find_all('div', attrs={'class': ['vacancy-serp-item__layout']})
    
    for v in vacancies:
        salary_min = 0
        salary_max = 0
        currency = ''
        vacancy_name = v.find('a', attrs={'class': ['serp-item__title']})
        v_name = vacancy_name.text
        v_link = vacancy_name['href']

        try:
            salary = v.find('span', attrs={'data-qa': ['vacancy-serp__vacancy-compensation']})
            s = re.match('[\d – ]+', salary.text)
            if s:
                min_max = s.group().strip().replace(' ', '').replace(' ', '')
                salary_min = int(min_max.split('–')[0])
                salary_max = int(min_max.split('–')[1])
            else:
                g = re.findall('[\d  ]+', salary.text)
                if g:
                    if 'до' in salary.text:
                        salary_max = int(g[0].replace(' ', '').replace(' ', ''))
                    elif 'от' in salary.text:
                        salary_min = int(g[0].replace(' ', '').replace(' ', ''))


            curr = re.findall('[А-яA-z]+\.*$', salary.text)
            if curr:
                currency = curr[0]
        except AttributeError:
            pass

        vacancies_list.append({
            'vacancy_name': v_name,
            'vacancy_link': v_link,
            'min_salary': salary_min,
            'max_salary': salary_max,
            'currency': currency
        })

In [148]:
df = pd.DataFrame(vacancies_list)

In [149]:
df

,vacancy_name,vacancy_link,min_salary,max_salary,currency
0,Data Scientist,https://kazan.hh.ru/vacancy/76925483?from=vaca...,160000,200000,руб.
1,Data scientist,https://kazan.hh.ru/vacancy/74376536?from=vaca...,200000,240000,руб.
2,Data Scientist (Middle+),https://kazan.hh.ru/vacancy/76810036?from=vaca...,170000,240000,руб.
3,Data Scientist (удаленно),https://kazan.hh.ru/vacancy/76655873?from=vaca...,0,0,
4,Data Scientist middle+/Senior,https://kazan.hh.ru/vacancy/77208642?from=vaca...,250000,350000,руб.
...,...,...,...,...,...
161,Senior Data Scientist (WAF),https://kazan.hh.ru/vacancy/74252169?from=vaca...,0,0,
162,Senior Data Scientist (WAF),https://kazan.hh.ru/vacancy/74252167?from=vaca...,0,0,
163,Team Lead Fraud Analyst,https://kazan.hh.ru/vacancy/75641735?from=vaca...,0,0,
164,Team Lead Fraud Analyst,https://kazan.hh.ru/vacancy/75641763?from=vaca...,0,0,


In [150]:
df.to_csv('vacancies.csv')